In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib  

In [9]:
# Define the Positional Encoding class
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_config(self):
        config = super().get_config().copy()
        return config

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            np.arange(position)[:, np.newaxis],
            np.arange(d_model)[np.newaxis, :],
            d_model)
        # Apply sin to even indices and cos to odd indices
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        return tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)

    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
        return pos * angle_rates

    def call(self, x):
        return x + self.pos_encoding[:, :tf.shape(x)[1], :]


In [10]:

# Define the Transformer Encoder layer
def transformer_encoder_layer(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Multi-Head Attention
    x = tf.keras.layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Network
    x = tf.keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(res)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res


In [11]:
# Load the model
model = tf.keras.models.load_model('transformer_chiller_model.h5', custom_objects={'PositionalEncoding': PositionalEncoding})

# Load the scaler
scaler = joblib.load('scaler.save')

TypeError: Error when deserializing class 'PositionalEncoding' using config={'name': 'positional_encoding', 'trainable': True, 'dtype': 'float32'}.

Exception encountered: PositionalEncoding.__init__() got an unexpected keyword argument 'name'